In [1]:
import torch 
from torch.utils.data import DataLoader
from src import full_pcd_dataset
import importlib
importlib.reload(full_pcd_dataset)
import json
# Create dataset instance
dataset = full_pcd_dataset.FullPCDDataset("data/full_pcd_300000_samples_6d.npz")
with open("data/label_dict.json", "r") as f:
    label_dict = json.load(f)
num_classes = len(label_dict.keys())
# Split into train and validation (example: 80-20 split)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Create data loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0,pin_memory=True)
num_classes

5

In [3]:
from src.model_updated import PointNet2Classification 
from torch.optim.lr_scheduler import StepLR
model = PointNet2Classification(num_classes, None)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5) 

In [4]:
from src.model_utils import count_parameters

print(f"Number of parameters in the model: {count_parameters(model)}")

Number of parameters in the model: 40613


In [5]:
from src.train_classificator import train_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
train_model(model, train_loader, val_loader, optimizer, scheduler, device, epochs=100, directory="cls_updated_vsmall_new_data")

Training: 100%|██████████| 1875/1875 [02:48<00:00, 11.11it/s]


Epoch 1/100 - Total Loss: 0.6323 | Accuracy: 66.28%


Validating: 100%|██████████| 469/469 [00:36<00:00, 12.82it/s]


Validation - Loss: 2.2988 | Accuracy: 60.39%


Training: 100%|██████████| 1875/1875 [02:54<00:00, 10.73it/s]


...

Training: 100%|██████████| 1875/1875 [02:51<00:00, 10.96it/s]


Epoch 34/100 - Total Loss: 0.0006 | Accuracy: 99.98%


Validating: 100%|██████████| 469/469 [00:34<00:00, 13.47it/s]


Validation - Loss: 45.1554 | Accuracy: 19.95%


Training:   8%|▊         | 147/1875 [00:12<02:26, 11.82it/s]